In [26]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px 
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [27]:
df = pd.read_excel(r'.\Data\AG_Demo_ihb.xlsx')
df.dropna(inplace=True)
df['call count percentage'] = df.groupby(['Reason', 'state', 'district', 'block_name'])['call count'].transform(lambda x: x / x.sum() * 100)
df.head()

,date,state,district,block_name,Reason,call count,call count percentage
0,2023-04-01,Bihar,BHAGALPUR,GAURADIH,Customer Not Interested,1,25.000000
1,2023-04-01,Assam,BARPETA,BARPETA ROAD,Customer doesn't want to download app,1,33.333333
2,2023-04-01,Assam,BAKSA,GHOGRACHWK,Customer already bought,1,100.000000
3,2023-04-01,Assam,NALBARI,NALBARI TOWN,Delivery charge high,1,100.000000
4,2023-04-01,Bihar,PATNA,PATNA SADAR,Customer already bought,1,11.111111


In [28]:
treemap_chart = px.treemap(df, path=['Reason', 'state', 'district', 'block_name'],
                           values='call count',
                           hover_data=['call count', 'call count percentage'],
                           labels={'call count': 'Call Count'},
                           title='IHB Not interested Reasons',
                           template='plotly')
treemap_chart.update_traces(textinfo='label+text+value+percent entry',marker_line_color='black',
                           marker_line_width=0.5)
treemap_chart.update_layout(
   uniformtext=dict(mode=False, minsize=8,),
)
treemap_chart.show()

In [31]:
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
# Assuming your DataFrame is already loaded and named 'df'
# Assuming 'date' column is in datetime format
# Create a treemap chart
treemap_chart = px.treemap(df, path=['Reason', 'state', 'district', 'block_name'],
                           values='call count',
                           hover_data=['call count', 'call count percentage'],
                           labels={'call count': 'Call Count'},
                           title='IHB Not Interested Reasons',
                           template='plotly',
                           color='Reason')
# Create Dash app
app = JupyterDash(__name__)
# Layout of the app
app.layout = html.Div(children=[
   dcc.Graph(id='treemap-graph', figure=treemap_chart),
])
# Callback to update the chart based on user input
@app.callback(
   Output('treemap-graph', 'figure'),
   Input('treemap-graph', 'clickData')
)
def update_chart(clickData):
   if clickData is None:
       # Initial state: show only the first layer
       return px.treemap(df, path=['Reason'],
                         values='call count',
                         hover_data=['call count', 'call count percentage'],
                         labels={'call count': 'Call Count'},
                         title='IHB Not Interested Reasons',
                         template='plotly',
                         color='Reason')
   else:
       # Update chart based on clicked data
       current_path = clickData['points'][0]['id']
       return px.treemap(df, path=current_path,
                         values='call count',
                         hover_data=['call count', 'call count percentage'],
                         labels={'call count': 'Call Count'},
                         title='IHB Not Interested Reasons',
                         template='plotly',
                         color=current_path[-1])
# Run the app
app.run_server(mode="inline")

Dash is running on http://127.0.0.1:8050/

